In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('mol2vec.csv')
df.replace(to_replace='#NAME?',value=np.nan,inplace=True)
df.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)
columns = df.columns
df = df.astype("float", errors = "ignore")
df = np.float32(df)
df[np.isnan(df)] = 0
df[np.isinf(df)] = 0
df=pd.DataFrame(df,columns=columns)
ncol = df.shape[1]-1
x = df.iloc[:,0:ncol]
y = df['active']
corr_matrix = x.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
x1 = x.drop(to_drop, axis=1)
x2 = x1.loc[:, (x1 != x1.iloc[0]).any()] 
x2 = x2.round(6)
import warnings
warnings.filterwarnings("ignore")

In [2]:
df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
0,-3.042170,-3.151222,-3.232676,7.560959,5.113686,-0.568981,-19.256578,-4.969994,16.522644,1.793195,...,12.655949,20.450172,4.086627,-10.276544,-2.190987,-1.115333,-9.388687,-15.132810,-4.057350,1.0
1,-4.727887,-4.351881,-4.658237,-1.792979,-0.499149,-8.724982,-8.886705,5.068110,-1.706909,-1.742328,...,14.233682,-0.425539,-0.464745,-10.994161,-12.039774,-12.723747,-4.175037,-11.076891,2.639213,1.0
2,-4.097823,-3.983348,-5.248144,-0.722321,-1.265023,-8.232691,-7.873246,5.745502,-0.900785,-1.008448,...,13.890897,-0.387179,0.128263,-11.509965,-10.279533,-11.952808,-3.907961,-10.316779,2.601157,1.0
3,-5.616269,-6.542370,-6.168895,-3.166161,1.509026,-11.890589,-10.365218,4.797748,-1.633804,-2.156420,...,16.483412,1.412684,-0.536551,-12.108947,-14.011013,-14.057915,-5.047316,-14.451323,1.707880,1.0
4,-4.396163,-5.507332,-5.355181,-2.798300,1.738860,-10.982598,-10.979731,4.270193,-1.071588,-2.923557,...,16.781738,-0.069494,-0.499126,-10.858052,-13.415128,-14.032432,-5.018421,-12.879906,1.733685,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,-0.245050,0.000723,-1.600506,2.734911,1.502080,0.887623,-5.741633,0.586250,4.324068,-0.374940,...,4.085058,5.202066,2.300924,-5.023169,-0.517164,-1.643741,-2.033808,-4.831588,0.884858,0.0
2020,-1.848185,-1.777678,-1.303566,0.651581,1.617647,1.635964,-2.599049,-0.967981,0.539179,1.095203,...,1.162264,3.851595,-0.020075,-1.346415,-1.949291,-0.575638,0.865116,-2.355083,-1.106393,0.0
2021,0.009402,0.458835,2.047133,-0.661602,1.160643,-0.072311,-4.937521,-2.287562,4.538611,-0.010992,...,1.278592,3.704251,0.990562,-2.375479,0.759892,-2.489954,-2.155487,-2.489589,-1.650008,0.0
2022,-3.098453,-1.974883,-2.323134,-0.688243,8.347803,0.660212,-16.231108,-3.275765,4.571203,-5.276364,...,3.041915,10.691944,3.452311,-7.664165,-4.948164,-1.304520,-8.855995,-8.327632,-5.991015,0.0


In [3]:
from sklearn.model_selection import train_test_split
x_train,x_val_test,y_train,y_val_test = train_test_split(x2,y,test_size = 0.2,random_state = 4)
x_val, x_test, y_val, y_test = train_test_split(x_val_test,y_val_test,test_size = 0.5,random_state = 4)

In [4]:
train = pd.concat([x_train,y_train],axis=1)
train

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
1410,0.597948,0.080038,-1.366101,2.647062,-1.487001,0.827820,-5.491724,0.602584,2.634875,1.991404,...,4.917458,4.469850,-1.766507,-3.387794,-0.513205,-1.663026,-1.801202,-3.240348,-1.202156,0.0
256,4.212898,-8.658149,-1.603910,7.160762,0.320550,0.002613,-18.824619,-1.000445,9.100834,3.340642,...,17.778799,12.607107,-2.537816,-13.025389,-5.962812,-10.513347,-8.907743,-17.662489,-3.014798,1.0
1433,0.663531,-0.199823,-1.753602,3.057977,0.404737,0.085814,-3.529144,0.033134,2.852578,1.091487,...,3.316037,3.877097,-0.061721,-2.441421,-0.297578,-1.267564,-0.533161,-3.561257,-0.246009,0.0
836,3.553452,-2.841779,-2.220558,7.534303,-0.470134,-0.670336,-12.446407,2.569174,4.584186,4.641965,...,11.852880,8.923352,-3.649912,-8.235695,-5.148130,-5.322622,-5.272404,-9.774830,0.662330,0.0
1519,-1.079556,-0.845350,-1.706922,-0.496573,4.482872,1.613211,-6.136358,-1.700262,5.521938,-4.268881,...,1.477869,3.940392,0.415042,-4.623921,-2.579385,-0.105511,-4.462537,-6.087024,-2.684478,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1921,0.567474,-0.528782,-1.294860,2.510436,-2.359934,0.912823,-4.673058,1.058856,2.897910,3.141804,...,5.159438,5.412770,-1.232882,-3.152432,0.028292,-1.415146,-1.729791,-2.353377,-0.412179,0.0
709,-2.737217,-3.061906,-4.850111,-0.970899,-0.184666,-9.712247,-7.118638,4.634256,-1.491924,-0.909947,...,13.895059,-0.277262,-0.264903,-11.864182,-12.195075,-12.304102,-3.580624,-11.380896,2.293994,1.0
439,8.606175,-9.303092,-2.818891,16.390532,0.366137,-1.115640,-16.178488,0.156895,4.188652,-0.228592,...,18.679991,11.153399,-5.769864,-9.819157,-11.202104,-11.058070,-7.712854,-18.904081,-3.582997,1.0
174,-3.158584,-16.545412,-1.918330,0.977628,-3.715431,-13.563825,-23.910465,2.265978,5.787884,-7.506930,...,23.277275,9.611921,-5.182190,-19.676712,-24.813051,-17.764711,-8.832652,-16.303173,6.217123,1.0


In [5]:
val = pd.concat([x_val,y_val],axis=1)
val

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
1313,-0.705401,-1.418557,-1.782935,1.871862,-1.644322,-2.198950,-6.589576,0.314057,3.881999,1.033322,...,5.354982,5.057994,-0.611369,-5.233748,-2.962978,-2.640161,-2.702640,-4.561736,0.998078,0.0
963,-4.567230,1.379578,-6.728546,-2.089763,4.931082,-3.899668,-10.930854,0.937629,1.661532,3.496615,...,4.371200,3.250466,-3.464776,-11.290627,-7.565286,-7.462164,-4.217870,-10.463884,-1.281795,1.0
253,-0.219577,-5.534134,-5.138458,5.852816,5.660736,0.633607,-12.388984,0.860317,6.906550,3.882762,...,10.564666,13.921894,-0.524491,-8.921689,-9.351679,-8.234758,-7.575264,-12.715349,-6.233711,1.0
1999,-0.986557,-1.979823,-1.269971,0.067183,2.995789,-4.632668,-4.702533,0.947612,1.100858,-0.505143,...,9.242188,-0.083746,1.211800,-6.792691,-3.661167,-4.185243,-3.297549,-7.267876,-2.130358,0.0
1652,1.839829,-2.206516,-0.326712,0.988506,-0.493589,0.411044,-0.818876,0.852043,-0.225296,-0.031272,...,3.204026,-0.392685,-0.923494,-2.789679,-1.376022,-2.539751,-1.100736,-0.634671,-0.301128,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,1.905233,-3.473932,-5.865212,5.729110,-1.074485,0.988830,-15.413540,-0.718564,5.680833,5.977876,...,14.775646,9.321992,-1.321805,-12.533912,-6.141512,-8.318372,-6.121054,-12.197614,-1.918811,1.0
1932,1.130856,-2.067951,-2.716505,0.642175,0.761404,2.476528,-10.015460,-1.877943,2.699778,3.333394,...,9.357099,7.393549,1.441057,-7.981624,-2.097896,-6.553251,-8.464458,-5.824588,-6.275820,0.0
1973,1.147677,-0.821721,-2.391354,2.878886,1.032573,0.759826,-4.913688,0.808204,4.581702,3.992270,...,4.914950,4.536654,-0.100607,-4.218724,-0.843059,-3.543380,-0.112206,-4.221433,-0.732344,0.0
1807,0.852751,-0.831783,-1.335856,2.651769,-2.625286,1.768429,-4.296264,1.109342,2.725805,3.751087,...,5.661756,6.285634,-1.371698,-3.295117,-0.384318,-2.134888,-1.586088,-2.733318,-0.988424,0.0


In [28]:
test1 = pd.concat([x_test,y_test],axis=1)
test1

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
1207,-1.059696,1.362882,-0.974827,1.007108,1.856092,-1.421007,-4.227500,-0.470202,0.844070,0.956002,...,1.154663,2.392234,-0.100949,-3.929020,-1.633472,-1.741680,-0.930012,-3.442616,-0.511529,0.0
1126,1.255841,-1.836725,-1.839920,5.054730,-0.555774,0.409044,-4.875106,-0.042847,3.896368,1.467310,...,4.469645,5.761068,-1.227115,-2.370480,-1.148394,-1.647679,-0.578619,-4.745800,-0.587339,0.0
301,0.422913,-2.265346,-5.919822,7.235750,2.199651,-0.818556,-10.099222,0.247358,4.453955,0.890357,...,11.843386,9.212858,-2.750053,-5.874560,-6.959916,-8.830362,-4.435438,-13.599303,-0.545961,1.0
1266,-0.853699,1.104774,-1.674481,-1.073425,1.516415,-1.843767,-1.881930,-0.651935,2.010216,0.059449,...,1.699111,2.051695,1.422203,-1.900405,-1.371449,-0.567273,-1.231166,-4.027981,-1.452958,0.0
1800,0.309878,-0.362497,-2.099364,3.619047,1.311080,-0.241741,-4.803797,-1.015429,3.711014,0.770010,...,4.198678,4.488040,0.343285,-3.029077,-1.016549,-1.646080,-1.310753,-4.257460,-1.455733,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330,0.365042,-1.349990,-1.693096,5.276032,-0.682004,0.260959,-5.502398,0.061947,4.785788,1.653456,...,4.993080,5.509728,-0.657402,-2.739898,-1.235318,-1.912304,-0.527374,-4.650898,-0.585187,0.0
1366,0.331824,-1.419044,-2.435758,3.218880,-0.316565,1.723669,-3.116811,0.269950,2.754530,3.461981,...,3.935123,5.828642,-0.108498,-3.525332,-2.013272,-2.076503,-0.991799,-4.220602,-1.951930,0.0
2016,0.708912,-1.009033,-2.156759,3.930840,0.640204,-0.058788,-5.483631,-0.404353,2.973248,1.503774,...,2.968716,4.633019,0.556773,-2.623432,-1.989006,-2.466921,-0.859673,-4.752112,-0.630234,0.0
707,4.689576,-7.776074,-2.771860,10.091646,2.986384,0.702584,-16.779085,-1.290133,6.438552,2.856995,...,13.476208,13.004099,-2.088026,-12.357860,-5.633462,-8.801695,-7.640206,-14.509857,-3.719482,1.0


In [7]:
col_name = train.columns.values.tolist()
col_name.pop()

'active'

In [8]:
from pycaret.classification import *
CM = setup(data = train, target = 'active', test_data = val, silent=True, numeric_features = col_name,
             session_id=0, fold_strategy = 'stratifiedkfold', fold = 5, fold_shuffle = True, normalize = True, normalize_method = 'minmax')

,Description,Value
0,session_id,0
1,Target,active
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(1619, 300)"
5,Missing Values,False
6,Numeric Features,299
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [9]:
mlp = create_model('mlp')
tunedmlp = tune_model(mlp,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedmlp)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8117,0.8870,0.7535,0.8045,0.7782,0.6149,0.6159
1,0.8549,0.9231,0.7887,0.8682,0.8266,0.7024,0.7048
2,0.8580,0.9133,0.8392,0.8392,0.8392,0.7121,0.7121
3,0.8519,0.9096,0.7902,0.8626,0.8248,0.6969,0.6989
4,0.8452,0.9017,0.7746,0.8594,0.8148,0.6824,0.6852
Mean,0.8443,0.9069,0.7893,0.8468,0.8167,0.6818,0.6834
Std,0.0169,0.0121,0.0282,0.0233,0.0208,0.0348,0.0349


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,MLP Classifier,0.8614,0.9184,0.8208,0.9062,0.8614,0.7235,0.7270


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.406483,0.851605,0.837115,0.337747,0.257631,0.755903,0.783518,0.596296,0.443665,0.599748,...,0.265846,0.834303,0.876184,0.883421,0.736536,0.892163,0.465810,0.0,0.0,0.9191
1,0.217867,0.922683,0.660512,0.197493,0.542049,0.710743,0.687070,0.629205,0.407694,0.684589,...,0.140649,0.690876,0.788379,0.782527,0.698915,0.788066,0.378407,1.0,1.0,0.7606
2,0.430211,0.747062,0.717292,0.478685,0.573610,0.831119,0.654675,0.625125,0.492662,0.697889,...,0.269658,0.746972,0.754297,0.766362,0.615554,0.748357,0.188568,1.0,1.0,0.8123
3,0.392751,0.837348,0.855432,0.273856,0.458338,0.691279,0.825441,0.629732,0.398611,0.546759,...,0.345839,0.797387,0.862863,0.851092,0.721765,0.844435,0.345876,0.0,0.0,0.9267
4,0.530795,0.831590,0.889115,0.306473,0.307406,0.825209,0.911723,0.624688,0.377128,0.563080,...,0.252151,0.892178,0.906461,0.885522,0.776310,0.961425,0.416003,0.0,0.0,0.9526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.533989,0.799395,0.691341,0.474305,0.282279,0.840551,0.587480,0.541798,0.472805,0.770050,...,0.234675,0.661435,0.815542,0.764613,0.651661,0.757488,0.353986,1.0,1.0,0.7693
198,0.496168,0.835109,0.803778,0.294212,0.361690,0.880055,0.707407,0.480611,0.424513,0.678968,...,0.355898,0.769233,0.892688,0.801545,0.593476,0.869890,0.186953,0.0,0.0,0.9159
199,0.496990,0.866766,0.815389,0.373399,0.373420,0.834470,0.820750,0.622375,0.455000,0.701661,...,0.288256,0.858339,0.916629,0.864522,0.800854,0.898165,0.399471,0.0,0.0,0.9481
200,0.482585,0.866511,0.853079,0.365358,0.215200,0.861252,0.834467,0.638267,0.424935,0.693354,...,0.232485,0.880210,0.925381,0.893993,0.764259,0.924411,0.389654,0.0,0.0,0.9336


In [10]:
xgb = create_model('xgboost')
tunedxgb = tune_model(xgb,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedxgb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8272,0.9001,0.8451,0.7792,0.8108,0.6522,0.6540
1,0.8519,0.9226,0.8380,0.8264,0.8322,0.6996,0.6996
2,0.8580,0.9295,0.8671,0.8212,0.8435,0.7138,0.7147
3,0.8457,0.9237,0.8252,0.8252,0.8252,0.6871,0.6871
4,0.8359,0.9104,0.8028,0.8201,0.8114,0.6662,0.6663
Mean,0.8437,0.9173,0.8356,0.8144,0.8246,0.6838,0.6843
Std,0.0110,0.0106,0.0213,0.0178,0.0125,0.0222,0.0219


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8812,0.9474,0.8774,0.8942,0.8857,0.7620,0.7622


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.406483,0.851605,0.837115,0.337747,0.257631,0.755903,0.783518,0.596296,0.443665,0.599748,...,0.265846,0.834303,0.876184,0.883421,0.736536,0.892163,0.465810,0.0,0.0,0.8834
1,0.217867,0.922683,0.660512,0.197493,0.542049,0.710743,0.687070,0.629205,0.407694,0.684589,...,0.140649,0.690876,0.788379,0.782527,0.698915,0.788066,0.378407,1.0,1.0,0.8891
2,0.430211,0.747062,0.717292,0.478685,0.573610,0.831119,0.654675,0.625125,0.492662,0.697889,...,0.269658,0.746972,0.754297,0.766362,0.615554,0.748357,0.188568,1.0,1.0,0.8429
3,0.392751,0.837348,0.855432,0.273856,0.458338,0.691279,0.825441,0.629732,0.398611,0.546759,...,0.345839,0.797387,0.862863,0.851092,0.721765,0.844435,0.345876,0.0,0.0,0.9392
4,0.530795,0.831590,0.889115,0.306473,0.307406,0.825209,0.911723,0.624688,0.377128,0.563080,...,0.252151,0.892178,0.906461,0.885522,0.776310,0.961425,0.416003,0.0,0.0,0.9723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.533989,0.799395,0.691341,0.474305,0.282279,0.840551,0.587480,0.541798,0.472805,0.770050,...,0.234675,0.661435,0.815542,0.764613,0.651661,0.757488,0.353986,1.0,1.0,0.9345
198,0.496168,0.835109,0.803778,0.294212,0.361690,0.880055,0.707407,0.480611,0.424513,0.678968,...,0.355898,0.769233,0.892688,0.801545,0.593476,0.869890,0.186953,0.0,0.0,0.7472
199,0.496990,0.866766,0.815389,0.373399,0.373420,0.834470,0.820750,0.622375,0.455000,0.701661,...,0.288256,0.858339,0.916629,0.864522,0.800854,0.898165,0.399471,0.0,0.0,0.9652
200,0.482585,0.866511,0.853079,0.365358,0.215200,0.861252,0.834467,0.638267,0.424935,0.693354,...,0.232485,0.880210,0.925381,0.893993,0.764259,0.924411,0.389654,0.0,0.0,0.9801


In [11]:
rf = create_model('rf')
tunedrf = tune_model(rf, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedrf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8086,0.9032,0.8310,0.7564,0.7919,0.6155,0.6179
1,0.8519,0.9299,0.8451,0.8219,0.8333,0.7000,0.7003
2,0.8457,0.9281,0.8601,0.8039,0.8311,0.6893,0.6907
3,0.8272,0.9237,0.8112,0.8000,0.8056,0.6500,0.6501
4,0.8421,0.9106,0.8169,0.8227,0.8198,0.6793,0.6793
Mean,0.8351,0.9191,0.8329,0.8010,0.8163,0.6668,0.6676
Std,0.0155,0.0104,0.0180,0.0241,0.0157,0.0306,0.0301


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8663,0.9452,0.8679,0.8762,0.8720,0.7321,0.7322


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.406483,0.851605,0.837115,0.337747,0.257631,0.755903,0.783518,0.596296,0.443665,0.599748,...,0.265846,0.834303,0.876184,0.883421,0.736536,0.892163,0.465810,0.0,0.0,0.7796
1,0.217867,0.922683,0.660512,0.197493,0.542049,0.710743,0.687070,0.629205,0.407694,0.684589,...,0.140649,0.690876,0.788379,0.782527,0.698915,0.788066,0.378407,1.0,1.0,0.8591
2,0.430211,0.747062,0.717292,0.478685,0.573610,0.831119,0.654675,0.625125,0.492662,0.697889,...,0.269658,0.746972,0.754297,0.766362,0.615554,0.748357,0.188568,1.0,1.0,0.7423
3,0.392751,0.837348,0.855432,0.273856,0.458338,0.691279,0.825441,0.629732,0.398611,0.546759,...,0.345839,0.797387,0.862863,0.851092,0.721765,0.844435,0.345876,0.0,0.0,0.8848
4,0.530795,0.831590,0.889115,0.306473,0.307406,0.825209,0.911723,0.624688,0.377128,0.563080,...,0.252151,0.892178,0.906461,0.885522,0.776310,0.961425,0.416003,0.0,0.0,0.9881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.533989,0.799395,0.691341,0.474305,0.282279,0.840551,0.587480,0.541798,0.472805,0.770050,...,0.234675,0.661435,0.815542,0.764613,0.651661,0.757488,0.353986,1.0,1.0,0.8277
198,0.496168,0.835109,0.803778,0.294212,0.361690,0.880055,0.707407,0.480611,0.424513,0.678968,...,0.355898,0.769233,0.892688,0.801545,0.593476,0.869890,0.186953,0.0,0.0,0.6523
199,0.496990,0.866766,0.815389,0.373399,0.373420,0.834470,0.820750,0.622375,0.455000,0.701661,...,0.288256,0.858339,0.916629,0.864522,0.800854,0.898165,0.399471,0.0,0.0,0.9326
200,0.482585,0.866511,0.853079,0.365358,0.215200,0.861252,0.834467,0.638267,0.424935,0.693354,...,0.232485,0.880210,0.925381,0.893993,0.764259,0.924411,0.389654,0.0,0.0,0.9844


In [12]:
et = create_model('et')
tunedet = tune_model(et, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedet)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8272,0.9068,0.8592,0.7722,0.8133,0.6533,0.6565
1,0.8611,0.9328,0.8521,0.8345,0.8432,0.7186,0.7187
2,0.8611,0.9304,0.8671,0.8267,0.8464,0.7198,0.7205
3,0.8457,0.9283,0.8392,0.8163,0.8276,0.6880,0.6882
4,0.8421,0.9148,0.8310,0.8138,0.8223,0.6803,0.6804
Mean,0.8474,0.9226,0.8497,0.8127,0.8306,0.6920,0.6928
Std,0.0128,0.0101,0.0131,0.0216,0.0125,0.0250,0.0242


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.8861,0.9493,0.8962,0.8879,0.8920,0.7716,0.7716


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.406483,0.851605,0.837115,0.337747,0.257631,0.755903,0.783518,0.596296,0.443665,0.599748,...,0.265846,0.834303,0.876184,0.883421,0.736536,0.892163,0.465810,0.0,0.0,0.7787
1,0.217867,0.922683,0.660512,0.197493,0.542049,0.710743,0.687070,0.629205,0.407694,0.684589,...,0.140649,0.690876,0.788379,0.782527,0.698915,0.788066,0.378407,1.0,1.0,0.9547
2,0.430211,0.747062,0.717292,0.478685,0.573610,0.831119,0.654675,0.625125,0.492662,0.697889,...,0.269658,0.746972,0.754297,0.766362,0.615554,0.748357,0.188568,1.0,1.0,0.8309
3,0.392751,0.837348,0.855432,0.273856,0.458338,0.691279,0.825441,0.629732,0.398611,0.546759,...,0.345839,0.797387,0.862863,0.851092,0.721765,0.844435,0.345876,0.0,0.0,0.9082
4,0.530795,0.831590,0.889115,0.306473,0.307406,0.825209,0.911723,0.624688,0.377128,0.563080,...,0.252151,0.892178,0.906461,0.885522,0.776310,0.961425,0.416003,0.0,0.0,0.9897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.533989,0.799395,0.691341,0.474305,0.282279,0.840551,0.587480,0.541798,0.472805,0.770050,...,0.234675,0.661435,0.815542,0.764613,0.651661,0.757488,0.353986,1.0,1.0,0.8294
198,0.496168,0.835109,0.803778,0.294212,0.361690,0.880055,0.707407,0.480611,0.424513,0.678968,...,0.355898,0.769233,0.892688,0.801545,0.593476,0.869890,0.186953,0.0,0.0,0.6826
199,0.496990,0.866766,0.815389,0.373399,0.373420,0.834470,0.820750,0.622375,0.455000,0.701661,...,0.288256,0.858339,0.916629,0.864522,0.800854,0.898165,0.399471,0.0,0.0,0.8908
200,0.482585,0.866511,0.853079,0.365358,0.215200,0.861252,0.834467,0.638267,0.424935,0.693354,...,0.232485,0.880210,0.925381,0.893993,0.764259,0.924411,0.389654,0.0,0.0,0.9724


In [13]:
catboost = create_model('catboost')
tunedcat1 = tune_model(catboost, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedcat1)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8333,0.9044,0.8380,0.7933,0.8151,0.6636,0.6644
1,0.8580,0.9264,0.8310,0.8429,0.8369,0.7112,0.7113
2,0.8549,0.9313,0.8601,0.8200,0.8396,0.7073,0.7080
3,0.8519,0.9285,0.8322,0.8322,0.8322,0.6996,0.6996
4,0.8421,0.9151,0.8169,0.8227,0.8198,0.6793,0.6793
Mean,0.8481,0.9211,0.8356,0.8222,0.8287,0.6922,0.6925
Std,0.0091,0.0100,0.0141,0.0165,0.0096,0.0181,0.0179


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,CatBoost Classifier,0.8812,0.9448,0.8774,0.8942,0.8857,0.7620,0.7622


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.406483,0.851605,0.837115,0.337747,0.257631,0.755903,0.783518,0.596296,0.443665,0.599748,...,0.265846,0.834303,0.876184,0.883421,0.736536,0.892163,0.465810,0.0,0.0,0.9438
1,0.217867,0.922683,0.660512,0.197493,0.542049,0.710743,0.687070,0.629205,0.407694,0.684589,...,0.140649,0.690876,0.788379,0.782527,0.698915,0.788066,0.378407,1.0,1.0,0.9650
2,0.430211,0.747062,0.717292,0.478685,0.573610,0.831119,0.654675,0.625125,0.492662,0.697889,...,0.269658,0.746972,0.754297,0.766362,0.615554,0.748357,0.188568,1.0,1.0,0.8924
3,0.392751,0.837348,0.855432,0.273856,0.458338,0.691279,0.825441,0.629732,0.398611,0.546759,...,0.345839,0.797387,0.862863,0.851092,0.721765,0.844435,0.345876,0.0,0.0,0.9496
4,0.530795,0.831590,0.889115,0.306473,0.307406,0.825209,0.911723,0.624688,0.377128,0.563080,...,0.252151,0.892178,0.906461,0.885522,0.776310,0.961425,0.416003,0.0,0.0,0.9918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.533989,0.799395,0.691341,0.474305,0.282279,0.840551,0.587480,0.541798,0.472805,0.770050,...,0.234675,0.661435,0.815542,0.764613,0.651661,0.757488,0.353986,1.0,1.0,0.9153
198,0.496168,0.835109,0.803778,0.294212,0.361690,0.880055,0.707407,0.480611,0.424513,0.678968,...,0.355898,0.769233,0.892688,0.801545,0.593476,0.869890,0.186953,0.0,0.0,0.8035
199,0.496990,0.866766,0.815389,0.373399,0.373420,0.834470,0.820750,0.622375,0.455000,0.701661,...,0.288256,0.858339,0.916629,0.864522,0.800854,0.898165,0.399471,0.0,0.0,0.9600
200,0.482585,0.866511,0.853079,0.365358,0.215200,0.861252,0.834467,0.638267,0.424935,0.693354,...,0.232485,0.880210,0.925381,0.893993,0.764259,0.924411,0.389654,0.0,0.0,0.9783


In [14]:
lightgbm = create_model('lightgbm')
tunedlightgbm = tune_model(lightgbm,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedlightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8272,0.9056,0.8239,0.7905,0.8069,0.6506,0.6510
1,0.8519,0.9279,0.8310,0.8310,0.8310,0.6991,0.6991
2,0.8735,0.9269,0.8671,0.8493,0.8581,0.7439,0.7441
3,0.8333,0.9230,0.8112,0.8112,0.8112,0.6620,0.6620
4,0.8359,0.9120,0.8028,0.8201,0.8114,0.6662,0.6663
Mean,0.8443,0.9191,0.8272,0.8204,0.8237,0.6844,0.6845
Std,0.0167,0.0088,0.0222,0.0196,0.0191,0.0339,0.0338


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8911,0.9506,0.8679,0.9200,0.8932,0.7823,0.7837


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.406483,0.851605,0.837115,0.337747,0.257631,0.755903,0.783518,0.596296,0.443665,0.599748,...,0.265846,0.834303,0.876184,0.883421,0.736536,0.892163,0.465810,0.0,0.0,0.9965
1,0.217867,0.922683,0.660512,0.197493,0.542049,0.710743,0.687070,0.629205,0.407694,0.684589,...,0.140649,0.690876,0.788379,0.782527,0.698915,0.788066,0.378407,1.0,1.0,0.9992
2,0.430211,0.747062,0.717292,0.478685,0.573610,0.831119,0.654675,0.625125,0.492662,0.697889,...,0.269658,0.746972,0.754297,0.766362,0.615554,0.748357,0.188568,1.0,1.0,0.9857
3,0.392751,0.837348,0.855432,0.273856,0.458338,0.691279,0.825441,0.629732,0.398611,0.546759,...,0.345839,0.797387,0.862863,0.851092,0.721765,0.844435,0.345876,0.0,0.0,0.9988
4,0.530795,0.831590,0.889115,0.306473,0.307406,0.825209,0.911723,0.624688,0.377128,0.563080,...,0.252151,0.892178,0.906461,0.885522,0.776310,0.961425,0.416003,0.0,0.0,0.9998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.533989,0.799395,0.691341,0.474305,0.282279,0.840551,0.587480,0.541798,0.472805,0.770050,...,0.234675,0.661435,0.815542,0.764613,0.651661,0.757488,0.353986,1.0,1.0,0.9841
198,0.496168,0.835109,0.803778,0.294212,0.361690,0.880055,0.707407,0.480611,0.424513,0.678968,...,0.355898,0.769233,0.892688,0.801545,0.593476,0.869890,0.186953,0.0,0.0,0.9627
199,0.496990,0.866766,0.815389,0.373399,0.373420,0.834470,0.820750,0.622375,0.455000,0.701661,...,0.288256,0.858339,0.916629,0.864522,0.800854,0.898165,0.399471,0.0,0.0,0.9996
200,0.482585,0.866511,0.853079,0.365358,0.215200,0.861252,0.834467,0.638267,0.424935,0.693354,...,0.232485,0.880210,0.925381,0.893993,0.764259,0.924411,0.389654,0.0,0.0,0.9997


In [25]:
from itertools import combinations
ml = [tunedrf, tunedet, tunedlightgbm, tunedcat1, tunedxgb,tunedmlp]
a2 =  list(combinations(ml, 2))
a3 =  list(combinations(ml, 3))
a4 =  list(combinations(ml, 4))
a5 =  list(combinations(ml, 5))
a6 =  list(combinations(ml, 6))
all = a2 + a3 + a4 + a5 + a6 
print(len(all))
print(len(a2),len(a3),len(a4),len(a5),len(a6))

57
15 20 15 6 1


In [16]:
base_score = 0
for i in all:
    blender = blend_models(i)
    predict_model(blender)
    test = pull()
    avg = test['Accuracy']
    avg = avg.iloc[0]
    if avg > base_score:
       base_score = avg
       best_params = i
print(best_params, base_score)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8241,0.9084,0.8380,0.7778,0.8068,0.6457,0.6472
1,0.8580,0.9262,0.8310,0.8429,0.8369,0.7112,0.7113
2,0.8673,0.9266,0.8741,0.8333,0.8532,0.7322,0.7329
3,0.8272,0.9238,0.8042,0.8042,0.8042,0.6495,0.6495
4,0.8421,0.9128,0.8028,0.8321,0.8172,0.6783,0.6787
Mean,0.8437,0.9196,0.8300,0.8181,0.8237,0.6834,0.6839
Std,0.0169,0.0075,0.0261,0.0239,0.0187,0.0339,0.0338


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8911,0.9462,0.8774,0.9118,0.8942,0.7821,0.7827


(<catboost.core.CatBoostClassifier object at 0x7f83dc608cd0>, XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, objective='binary:logistic',
              predictor='auto', random_state=0, reg_alpha=0, ...), MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
            

In [26]:
base_score = 0
for i in all:
    blender = blend_models(i)
    predict_model(blender)
    test = pull()
    avg = test['AUC']
    avg = avg.iloc[0]
    if avg > base_score:
       base_score = avg
       best_params = i
print(best_params, base_score)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8333,0.9049,0.8380,0.7933,0.8151,0.6636,0.6644
1,0.8611,0.9312,0.8310,0.8489,0.8399,0.7173,0.7174
2,0.8642,0.9291,0.8601,0.8367,0.8483,0.7254,0.7256
3,0.8519,0.9258,0.8182,0.8417,0.8298,0.6987,0.6989
4,0.8390,0.9151,0.8099,0.8214,0.8156,0.6728,0.6728
Mean,0.8499,0.9212,0.8314,0.8284,0.8297,0.6955,0.6958
Std,0.0121,0.0099,0.0174,0.0197,0.0131,0.0241,0.0240


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8861,0.9474,0.8774,0.9029,0.8900,0.7721,0.7724


(ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight='balanced',
                     criterion='entropy', max_depth=11,
                     max_features=0.65515454980115, max_leaf_nodes=None,
                     max_samples=None,
                     min_impurity_decrease=2.1080969702098964e-09,
                     min_impurity_split=None, min_samples_leaf=1,
                     min_samples_split=7, min_weight_fraction_leaf=0.0,
                     n_estimators=147, n_jobs=-1, oob_score=False,
                     random_state=0, verbose=0, warm_start=False), LGBMClassifier(bagging_fraction=0.7118467713670378, bagging_freq=2,
               boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.8619532906931409, importance_type='split',
               learning_rate=0.026260304633362635, max_depth=-1,
               min_child_samples=1, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=300, n_jobs=-1, nu

In [29]:
blend = blend_models([tunedxgb,tunedlightgbm,tunedet])
md = predict_model(blend,data = test1)
md
y_pred = md['Label']
y_true = md['active']
t = y_pred.astype(float)
t
from sklearn.metrics import balanced_accuracy_score
accuracy = balanced_accuracy_score(y_true, t)
accuracy

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8272,0.9059,0.8380,0.7829,0.8095,0.6517,0.6529
1,0.8488,0.9292,0.8310,0.8252,0.8281,0.6931,0.6931
2,0.8642,0.9294,0.8601,0.8367,0.8483,0.7254,0.7256
3,0.8364,0.9260,0.8182,0.8125,0.8153,0.6685,0.6685
4,0.8328,0.9137,0.8028,0.8143,0.8085,0.6602,0.6602
Mean,0.8419,0.9208,0.8300,0.8143,0.8219,0.6798,0.6801
Std,0.0132,0.0094,0.0192,0.0180,0.0149,0.0267,0.0265


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8374,0.9134,0.7857,0.8148,0.8000,0.6631,0.6635


0.8298319327731092

In [27]:
test

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8861,0.9474,0.8774,0.9029,0.89,0.7721,0.7724
